# Simple Rating
* Predicts the global mean across all users and items

In [ ]:
medium = "";

In [ ]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../Alpha.ipynb");

In [ ]:
const metric = "drop"
const name = "$medium/$metric/Simple"
set_logging_outdir(name);

## Training

In [ ]:
function get_drop_rate(split, metric, medium)
    df = get_split(split, metric, medium, [:itemid, :metric])
    counts = Dict{Int32,Int32}()
    drops = Dict{Int32,Int32}()
    @showprogress for i = 1:length(df.itemid)
        u = df.itemid[i]
        if u ∉ keys(counts)
            counts[u] = 0
            drops[u] = 0
        end
        counts[u] += 1
        drops[u] += df.metric[i]
    end
    drop_rate = Dict{Int32,Float32}()
    for k in keys(counts)
        drop_rate[k] = drops[k] / counts[k]
    end
    drop_rate
end;

In [ ]:
const drop_rate = get_drop_rate("training", metric, medium);

## Inference

In [ ]:
function make_prediction(users, items, p)
    r = zeros(Float32, length(users))
    @showprogress for i = 1:length(r)
        if items[i] in keys(p)
            r[i] = p[items[i]]
        end
    end
    r
end;

In [ ]:
write_params(Dict("drop_rate" => drop_rate), name);

In [ ]:
model(users, items) = make_prediction(users, items, drop_rate);

In [ ]:
write_alpha(model, medium, name, ["training", "validation", "test", "negative"]);

In [ ]:
for split in ["validation", "test"]
    val = compute_loss(metric, medium, [name], split)
    @info "$split loss = $val"
end;